In [ ]:
from datetime import datetime
import os
import pandas as pd
import re
import time
from IPython.display import clear_output, display
import pickle

In [ ]:
EXPERIMENT_ID = 'circymxx' # Enter the ID of the experiment (name of the folder) to export
interval = 60 # in seconds for monitoring

In [ ]:
def experiment_to_csv(path, experiment):
    jobs = pd.read_csv(f'{experiment.path}/design.csv', sep='\t')
    jobs.columns.values[0] = 'Index'
    jobs = jobs[jobs.Index >= 0]
    
    jobs['accuracy'] = 0.0
    jobs['epochs'] = 0
    pat = re.compile("accuracy: (\d+\.\d+)")
    pat_clock = re.compile("Wall clock time is (\d+\.\d+) ms")
    found, not_existing = 0, 0
    for idx, row in jobs.iterrows():
        filename = f"{int(row.Index)}-model{row.model}-epochs{row.max_epochs}-executor_memory{row.executor_memory}-executor_cores{row.executor_cores}.log"
        
        try:
            with open(f'{experiment.path}/{filename}', 'r') as file:
                content = file.read()
                result = pat.findall(content)
                result_wallclock = float(pat_clock.findall(content)[-1])/1000
                jobs.at[idx, 'time'] = result_wallclock
                jobs.at[idx, 'accuracy'] = result[-1] if len(result) > 0 else 0
                jobs.loc[idx, 'epochs'] = len(result)
                found += 1
        except FileNotFoundError as e:
            jobs.at[idx, 'time'] = -1
            jobs.at[idx, 'accuracy'] = -1
            jobs.loc[idx, 'epochs'] = -1
            not_existing += 1
    
    time = os.stat(f'{experiment.path}/design.csv').st_mtime
    dt = datetime.fromtimestamp(time)
    jobs.to_csv(f'ex-{dt.strftime("%Y-%m-%d_%H:%M")}.csv')
    return (jobs, f'ex-{dt.strftime("%Y-%m-%d_%H:%M")}.csv', found, not_existing)

In [ ]:
def fetch_results():
    dir = os.scandir('raw/')
    experiments = list(filter(lambda x: x.is_dir() and x.name[0] != '.' and x.name == EXPERIMENT_ID, dir))
    assert len(experiments) > 0, f'The folder {EXPERIMENT_ID} does not exist!'

    for experiment in experiments:
        if os.path.exists(f'{experiment.path}/design.csv'):
            jobs, path, found, not_existing = experiment_to_csv(experiment.path, experiment)
            print(f'Results of {experiment} stored in {path}')
            print(f'{found} of the {found + not_existing} expected log files are present. The other experiments are probably missing.\n')
            df = pd.read_csv(path)
            df = df[['max_epochs', 'executor_memory', 'executor_cores', 'model', 'accuracy', 'time']]
        else:
            raise Exception(f'{experiment.path}/design.csv not found!')
        
        if 
    done = not_existing == 0
    return df, done

In [ ]:
df, _ = fetch_results()
df

In [ ]:
done = False
while not done:
    df, done = fetch_results()
    clear_output(wait=True)
    dt = datetime.fromtimestamp(time.time())
    print(f'Last update: {dt.strftime("%H:%M:%S")}')
    display(df)
    time.sleep(interval)
    
with open(f'data/{EXPERIMENT_ID}.pickle', 'wb') as handle:
    pickle.dump(df, handle)

# ANOVA

In [175]:
import os
import researchpy as rp

import statsmodels.api as sm
from statsmodels.formula.api import ols

dir = os.scandir('data/')
experiments = list(filter(lambda x: x.name.endswith(".pickle"), dir))

df = pd.DataFrame()

for experiment in experiments:
    dffile = open(experiment.path, 'rb')      
    read_df = pickle.load(dffile)
    read_df = read_df[['max_epochs', 'executor_memory', 'executor_cores', 'model', 'accuracy', 'time']]
    df = pd.concat([df, read_df])

        
acc_df = pd.DataFrame({'y': df["accuracy"],
                       'epochs': df["max_epochs"],
                       'memory': df["executor_memory"],
                       'cores': df["executor_cores"],
                       'model': df["model"]})

time_df = pd.DataFrame({'y': df["time"],
                        'epochs': df["max_epochs"],
                        'memory': df["executor_memory"],
                        'cores': df["executor_cores"],
                        'model': df["model"]})

print(f"Imported {len(experiments)} experiments!")

Imported 5 experiments!


## DF Summaries

In [176]:
summary_acc_df = rp.summary_cont(acc_df.groupby(['model', 'epochs', 'cores', 'memory']))['y']
summary_acc_df

N    Mean      SD      SE  95% Conf.  Interval
model  epochs cores memory                                                
bi-rnn 20     8     4       5  0.3995  0.0106  0.0048     0.3863    0.4127
                    32      5  0.4163  0.0203  0.0091     0.3911    0.4416
              16    4       5  0.4103  0.0139  0.0062     0.3931    0.4275
                    32      5  0.4074  0.0075  0.0033     0.3981    0.4167
       60     8     4       5  0.7198  0.0334  0.0149     0.6784    0.7613
                    32      5  0.7261  0.0327  0.0146     0.6855    0.7668
              16    4       5  0.7159  0.0595  0.0266     0.6420    0.7897
                    32      5  0.7497  0.0336  0.0150     0.7080    0.7914
lenet5 20     8     4       5  0.9323  0.0023  0.0010     0.9294    0.9351
                    32      5  0.9351  0.0034  0.0015     0.9309    0.9392
              16    4       5  0.9356  0.0063  0.0028     0.9278    0.9434
                    32      5  0.9338  0.0009  0.0004     0.9326    0.9349
       60     8     4       5  0.9584  0.0014  0.0006     0.9567    0.9601
                    32      5  0.9604  0.0013  0.0006     0.9589    0.9620
              16    4       5  0.9596  0.0022  0.0010     0.9568    0.9623
                    32      5  0.9599  0.0034  0.0015     0.9556    0.9641

In [177]:
summary_time_df = rp.summary_cont(time_df.groupby(['model', 'epochs', 'cores', 'memory']))['y']
summary_time_df

N       Mean        SD       SE  95% Conf.  \
model  epochs cores memory                                               
bi-rnn 20     8     4       5   347.3691   28.0840  12.5596   312.4982   
                    32      5   345.7391   20.0969   8.9876   320.7855   
              16    4       5   308.6550   34.6991  15.5179   265.5704   
                    32      5   308.1720   38.2285  17.0963   260.7050   
       60     8     4       5   996.7127   64.3048  28.7580   916.8677   
                    32      5  1008.8503   65.0580  29.0948   928.0701   
              16    4       5   888.7607   91.2986  40.8300   775.3985   
                    32      5   895.5210  114.3337  51.1316   753.5570   
lenet5 20     8     4       5   174.9143   11.3552   5.0782   160.8149   
                    32      5   175.6735   10.5155   4.7027   162.6167   
              16    4       5   172.4267   12.4227   5.5556   157.0019   
                    32      5   173.4769   12.0750   5.4001   158.4838   
       60     8     4       5   498.6016   28.5953  12.7882   463.0958   
                    32      5   499.8644   29.9425  13.3907   462.6859   
              16    4       5   492.1793   30.3675  13.5808   454.4731   
                    32      5   494.0800   28.4499  12.7232   458.7548   

                             Interval  
model  epochs cores memory             
bi-rnn 20     8     4        382.2400  
                    32       370.6927  
              16    4        351.7397  
                    32       355.6390  
       60     8     4       1076.5576  
                    32      1089.6304  
              16    4       1002.1229  
                    32      1037.4850  
lenet5 20     8     4        189.0136  
                    32       188.7302  
              16    4        187.8515  
                    32       188.4700  
       60     8     4        534.1074  
                    32       537.0429  
              16    4        529.8855  
                    32       529.4052

## ANOVA analysis

In [178]:
def anova_analysis(input_df):
    model = ols('y ~ C(model)*C(epochs)*C(cores)*C(memory)', input_df).fit()
    
    res = sm.stats.anova_lm(model, typ=2)
    res['PR(>F) < 0.05'] = res['PR(>F)'] < 0.05
    return res

In [179]:
acc_res = anova_analysis(acc_df)
print("Accuracy ANOVA Analysis")
acc_res

Accuracy ANOVA Analysis


,sum_sq,df,F,PR(>F),PR(>F) < 0.05
C(model),2.868766e+00,1.0,5961.048481,6.893245e-65,True
C(epochs),5.947628e-01,1.0,1235.865788,1.448051e-43,True
C(cores),1.839211e-04,1.0,0.382172,5.386369e-01,False
C(memory),1.030330e-03,1.0,2.140937,1.483084e-01,False
C(model):C(epochs),4.325476e-01,1.0,898.796544,2.169089e-39,True
C(model):C(cores),1.121011e-04,1.0,0.232936,6.310017e-01,False
C(epochs):C(cores),8.302813e-05,1.0,0.172525,6.792665e-01,False
C(model):C(memory),8.070851e-04,1.0,1.677053,1.999688e-01,False
C(epochs):C(memory),2.383951e-04,1.0,0.495364,4.840970e-01,False
C(cores):C(memory),5.611250e-07,1.0,0.001166,9.728668e-01,False


In [180]:
time_res = anova_analysis(time_df)
print("Time ANOVA Analysis")
time_res

Time ANOVA Analysis


,sum_sq,df,F,PR(>F),PR(>F) < 0.05
C(model),1.827953e+06,1.0,777.463445,1.623388e-37,True
C(epochs),4.437158e+06,1.0,1887.208631,3.252469e-49,True
C(cores),3.090026e+04,1.0,13.142477,5.729783e-04,True
C(memory),1.479361e+02,1.0,0.062920,8.027424e-01,False
C(model):C(epochs),4.437783e+05,1.0,188.747436,9.335449e-21,True
C(model):C(cores),2.461767e+04,1.0,10.470369,1.921910e-03,True
C(epochs):C(cores),7.269732e+03,1.0,3.091957,8.346086e-02,False
C(model):C(memory),4.360082e+01,1.0,0.018544,8.921085e-01,False
C(epochs):C(memory),1.563117e+02,1.0,0.066482,7.973563e-01,False
C(cores):C(memory),3.406068e+00,1.0,0.001449,9.697572e-01,False
